In [18]:
import pandas as pd
import json
import networkx
import time

In [19]:
W = pd.read_csv("../data/dual-chart.csv")
W.set_index("Attacking", inplace = True)
nodes = W.index.to_list()

In [20]:
W_pruned = W.copy()

threshold = 1.5

for A in nodes:
    for B in nodes:
        if A == B:
            continue

        difference = abs(W.loc[A,B]-W.loc[B,A])

        if difference < threshold:

            W_pruned.loc[A,B] = 0
            W_pruned.loc[B,A] = 0

In [21]:
t1 = time.time()
G_unpruned = networkx.from_pandas_adjacency(W)
triangles_unpruned = sum(networkx.triangles(G_unpruned).values())/3
t2 = time.time()

t3 = time.time()
G = networkx.from_pandas_adjacency(W_pruned)
triangles_pruned = sum(networkx.triangles(G).values())/3
t4 = time.time()

print(f"Unpruned Triangles : {triangles_unpruned}\nPruned Triangles : {triangles_pruned}")

Unpruned Triangles : 585276.0
Pruned Triangles : 870.0


In [22]:
print(f"Pruned time : {t4-t3}\nUnpruned time : {t2-t1}\nPercentage decrease = {100-(t4-t3)/(t2-t1)}")

Pruned time : 0.005364418029785156
Unpruned time : 0.08997750282287598
Percentage decrease = 99.94038045220765


In [23]:
triangles = set()

for u in G:
    for v in G.neighbors(u):

        if v <= u:
            continue
        
        for w in G.neighbors(v):

            if w <= v:
                continue
            
            if G.has_edge(u,w):
                triangles.add((u,v,w))

triangles = list(triangles)

In [24]:
with open("../data/triangles.json", "w") as file:
    json.dump(triangles, file, indent = 2)